<a href="https://colab.research.google.com/github/viktorjovev/conformal_prediction_regression_modeling/blob/main/conf_pred_birth_weight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
kagglehub.login()

prediction_interval_competition_i_birth_weight_path = kagglehub.competition_download('prediction-interval-competition-i-birth-weight')

print('Data source import complete.')


In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import os
for dirname, _, filenames in os.walk(prediction_interval_competition_i_birth_weight_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
def fast_mwis_score(y_true, lower, upper, alpha):
    assert y_true.ndim == 1, "y_true: pandas Series or 1D array expected"
    assert lower.ndim == 1, "lower: pandas Series or 1D array expected"
    assert upper.ndim == 1, "upper: pandas Series or 1D array expected"
    assert isinstance(alpha, float), "alpha: float expected"
    assert (lower <= upper).all(), ("lower must be <= upper",
                                    lower[lower > upper],
                                    upper[lower > upper])

    total_interval_width = upper.sum() - lower.sum()
    error_above = (y_true - upper)[y_true > upper].sum()
    error_below = (lower - y_true)[y_true < lower].sum()
    total_error = error_above + error_below
    mwis = (total_interval_width + total_error * 2 / alpha) / len(y_true)
    below = (y_true < lower).mean()
    above = (upper < y_true).mean()
    coverage = ((lower <= y_true) & (y_true <= upper)).mean()
    return mwis, below, coverage, above

In [ ]:
!pip install mapie

In [ ]:
df = pd.read_csv("/root/.cache/kagglehub/competitions/prediction-interval-competition-i-birth-weight/train.csv")
df['LD_INDL'] = df['LD_INDL'].map({'N':0, 'Y': 1})
df['RF_CESAR'] = df['RF_CESAR'].map({'N':0, 'Y': 1})
df['SEX'] = df['SEX'].map({'F':0, 'M': 1})
df['DMAR'] = df['DMAR'].map({'1': 'Married', '2': 'Unmarried', ' ': 'Not_declared'})
df = pd.concat([df.drop(columns = 'DMAR', axis = 1), pd.get_dummies(df['DMAR'], dtype = int).drop(columns = ['Not_declared'], axis = 1)], axis = 1)
df['approx_term'] = np.where(df['DLMP_MM'] != 99, df['DOB_MM'] - df['DLMP_MM'], 0)
df['approx_term'] = np.where(df['approx_term'] < 0,
                                 df['approx_term'] + 12,
                                 df['approx_term'])
df['plural_delivery'] = (df['ILLB_R'] == 3).astype(float)
df = df.drop(columns = ['id', 'DOB_TT', 'DOB_MM'], axis = 1)


In [ ]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)
x = df['DBWT'].dropna()
bins = np.linspace(x.min(), x.max(), 30)
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(x, bins=bins, alpha=0.5, color='blue')
ax.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray')
ax.set_title('Распределба на Тежина на новороденчињата', fontsize=14)
ax.set_xlabel('Тежина (грамови)', fontsize=12)
ax.set_ylabel('Број на новороденчиња', fontsize=12)
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.boxplot(df['DBWT'].dropna())
plt.title('Отстапувања во тежините на новороденчињата', fontsize=14)
plt.ylabel('Тежина (грамови)', fontsize=12)
plt.xlabel('', fontsize=1)

plt.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray')  #
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

plt.scatter(range(len(df['DBWT'])), df['DBWT'], alpha=0.2, color='blue', label='')

threshold = 1.5
outliers = df[(df['DBWT'] < df['DBWT'].quantile(0.25) - threshold * (df['DBWT'].quantile(0.75) - df['DBWT'].quantile(0.25))) |
              (df['DBWT'] > df['DBWT'].quantile(0.75) + threshold * (df['DBWT'].quantile(0.75) - df['DBWT'].quantile(0.25)))]

plt.scatter(outliers.index, outliers['DBWT'],  alpha=0.7 ,color='red', label='Отстапувања')
plt.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray')
plt.legend()
plt.show()


In [ ]:
import seaborn as sns

correlation_matrix = df._get_numeric_data().corr()
sns.heatmap(correlation_matrix, annot=False, cmap="coolwarm")
plt.title("Корелациона матрица")
plt.show()

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform

def distance(data):
    corr = data.corr(method = 'spearman')
    dist_linkage = linkage(squareform(1 - abs(corr)), 'complete')
    plt.figure(figsize = (8, 3), dpi = 500)
    dendro = dendrogram(dist_linkage, labels=data.columns, leaf_rotation=90)
    plt.title(f'Дендограм')
    plt.show()

distance(df._get_numeric_data())

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedStratifiedKFold, cross_val_score, cross_val_predict, RepeatedKFold, cross_validate
skf = KFold(n_splits = 5,  random_state = 42, shuffle= True)

In [ ]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from mapie.regression import MapieRegressor, MapieQuantileRegressor
X = df.drop(columns=['DBWT'], axis=1)
y = df['DBWT']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
ridge = Ridge(alpha=1.0)
ridge.fit(X_train,y_train)
mapie = MapieRegressor(estimator=ridge, cv=skf, method='naive')
mapie.fit(X_train, y_train)

In [ ]:
mapie.conformity_scores_.mean()

In [ ]:
np.set_printoptions(suppress=True)
coefficients = np.round(ridge.coef_, 3)
coefficients

In [ ]:
intercept = np.round(ridge.intercept_, 3)
intercept

In [ ]:
y_train_preds = []
y_train_pis = []

n_batches = len(X_train) // 10000

for X_train_batch in np.array_split(X_train, n_batches):
    y_preds_batch, y_pis_batch = mapie.predict(X_train_batch, alpha=0.1)
    y_train_preds.append(y_preds_batch)
    y_train_pis.append(y_pis_batch)

In [ ]:
lower_train_bounds = np.concatenate([array[:, 0, 0] for array in y_train_pis])
upper_train_bounds = np.concatenate([array[:, 1, 0] for array in y_train_pis])
y_train_preds = np.concatenate([array[:] for array in y_train_preds])


print("Lower bounds:", lower_train_bounds)
print("Upper bounds:", upper_train_bounds)

In [ ]:
fast_mwis_score(y_train, lower_train_bounds, upper_train_bounds, alpha = 0.1)

In [ ]:
plt.figure(figsize=(10, 4))
plt.hist(lower_train_bounds, bins=np.arange(100, 5100, 20), label='долна')
plt.hist(upper_train_bounds, bins=np.arange(100, 5100, 20), label='горна')
plt.xlabel('граница', fontsize=12)
plt.ylabel('број на граници')
plt.legend()
plt.show()

In [ ]:
sample_size = int(0.3 * len(y_train))
indices = np.random.choice(len(y_train), size=sample_size, replace=False)

y_train_numpy = y_train.to_numpy().flatten()
y_train_sample = y_train_numpy[indices]
y_train_preds_sample = y_train_preds[indices]
lower_bounds_sample = lower_train_bounds[indices]
upper_bounds_sample = upper_train_bounds[indices]

inside_interval = (y_train_sample >= lower_bounds_sample) & (y_train_sample <= upper_bounds_sample)
outside_interval = ~inside_interval

plt.figure(figsize=(8, 6))

plt.errorbar(
    y_train_sample[inside_interval],
    y_train_preds_sample[inside_interval],
    yerr=[
        y_train_preds_sample[inside_interval] - lower_bounds_sample[inside_interval],
        upper_bounds_sample[inside_interval] - y_train_preds_sample[inside_interval],
    ],
    fmt='o',
    ecolor='blue',
    alpha=0.7,
    label='Во рамки на предвидениот интервал',
)

plt.errorbar(
    y_train_sample[outside_interval],
    y_train_preds_sample[outside_interval],
    yerr=[
        y_train_preds_sample[outside_interval] - lower_bounds_sample[outside_interval],
        upper_bounds_sample[outside_interval] - y_train_preds_sample[outside_interval],
    ],
    fmt='o',
    ecolor='red',
    alpha=0.1,
    label='Грешни предвидувања',
)

plt.plot(
    y_train_sample,
    y_train_sample,
    color="black", linestyle="--", linewidth=1, label="Референтни предвидувања"
)

plt.xlabel('Референтни предвидувања', fontsize=12)
plt.ylabel('Предвидени вредности', fontsize=12)
plt.legend(fontsize=10)
plt.grid(alpha=0.3)
plt.show()

In [ ]:
residuals = y_train.to_numpy() - y_train_preds
temp = pd.DataFrame({'y_true': y_train.to_numpy(),
                     'y_pred': y_train_preds,
                     'residual': residuals})
temp['squared_residual'] = np.square(temp['residual'])
temp.sort_values('y_pred', inplace=True)
temp_g = temp.groupby(pd.qcut(temp.y_pred, 1000), observed=True).mean()

plt.figure(figsize=(8, 2))
plt.hist(temp.residual, bins=100, density=True)
plt.xlabel('резидуал')
plt.ylabel('густина')
plt.show()

ymin = df['DBWT'].min()

plt.figure(figsize=(8, 6))
plt.scatter(y_train_preds, residuals, s=10, c='g', alpha=0.7)
plt.fill_between(temp_g.y_pred,
                 - np.sqrt(temp_g.squared_residual),
                 + np.sqrt(temp_g.squared_residual),
                 color='gray', alpha=0.5, label='± Стандардна девијација')
plt.axhline(0, color='k', linestyle='--', linewidth=1)
plt.xlabel('Предвидувања')
plt.ylabel('Резидуали')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(16, 4))
plt.scatter(np.arange(len(X_train)), temp.residual, s=1, c='g', alpha=0.7)
plt.axhline(0, color='k')
plt.xlabel('index')
plt.ylabel('residual')
plt.show()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
degree = 2
ridge_alpha = 1.0

model = make_pipeline(
    PolynomialFeatures(degree),
    Ridge(alpha=ridge_alpha)
)

mapie = MapieRegressor(estimator=model, cv=skf, method='naive')
mapie.fit(X_train, y_train)

In [ ]:
y_train_preds = []
y_train_pis = []

# Predict with confidence intervals (e.g., 90% confidence level)
n_batches = len(X_train) // 10000

for X_train_batch in np.array_split(X_train, n_batches):
    y_preds_batch, y_pis_batch = mapie.predict(X_train_batch, alpha=0.1)
    y_train_preds.append(y_preds_batch)
    y_train_pis.append(y_pis_batch)

In [ ]:
lower_train_bounds = np.concatenate([array[:, 0, 0] for array in y_train_pis])
upper_train_bounds = np.concatenate([array[:, 1, 0] for array in y_train_pis])
y_train_preds = np.concatenate([array[:] for array in y_train_preds])


print("Lower bounds:", lower_train_bounds)
print("Upper bounds:", upper_train_bounds)

In [ ]:
fast_mwis_score(y_train, lower_train_bounds, upper_train_bounds, alpha = 0.1)

In [ ]:
n_batches = len(X_test) // 10000
y_preds = []
y_pis = []

for X_test_batch in np.array_split(X_test, n_batches):
    y_preds_batch, y_pis_batch = mapie.predict(X_test_batch, alpha=0.1)
    y_preds.append(y_preds_batch)
    y_pis.append(y_pis_batch)

In [ ]:
lower_bounds = np.concatenate([array[:, 0, 0] for array in y_pis])
upper_bounds = np.concatenate([array[:, 1, 0] for array in y_pis])
y_preds = np.concatenate([array[:] for array in y_preds])

print("Lower bounds:", lower_bounds)
print("Upper bounds:", upper_bounds)

In [ ]:
fast_mwis_score(y_test, lower_bounds, upper_bounds, alpha = 0.1)

In [ ]:
plt.figure(figsize=(10, 4))
plt.hist(lower_bounds, bins=np.arange(100, 5100, 20), label='долна граница')
plt.hist(upper_bounds, bins=np.arange(100, 5100, 20), label='горна граница')

plt.legend()
plt.show()